In [2]:
"""STEADY-STATE PARAMETERS"""

import numpy as np

DENSITY = 880.0  # kg/m^3
G = 9.81  # m/s^2
FRICTION_FACTOR = 0.02
PIPE_LENGTH = 10.0  # m
PIPE_DIAMETER = 0.1  # m
PIPE_AREA = np.pi * (PIPE_DIAMETER / 2.0) ** 2
STATIC_HEAD = DENSITY * G * 10.0
NUM_OF_PUMPS = 2

In [3]:
"""NETWORK GEOMETRY"""

import numpy as np


N_NODES = 4
N_BRANCHES = 5
N_CONTROL = 3

INTERFACE_MATRIX = np.zeros((N_NODES, N_BRANCHES))
INTERFACE_MATRIX[0, 0] = -1
INTERFACE_MATRIX[1, 0] = 1
INTERFACE_MATRIX[1, 1] = -1
INTERFACE_MATRIX[2, 1] = 1
INTERFACE_MATRIX[2, 2] = -1
INTERFACE_MATRIX[3, 2] = 1
INTERFACE_MATRIX[0, 3] = 1
INTERFACE_MATRIX[3, 3] = -1
INTERFACE_MATRIX[0, 4] = 1
INTERFACE_MATRIX[3, 4] = -1
print(INTERFACE_MATRIX)

[[-1.  0.  0.  1.  1.]
 [ 1. -1.  0.  0.  0.]
 [ 0.  1. -1.  0.  0.]
 [ 0.  0.  1. -1. -1.]]


In [4]:
"""NONLINEAR TERMS"""

from utils import darcy_weisbach_pressure_drop, valve_pressure_drop, pump_pressure_rise


def pump_source_dp(q, pump_speed, num_of_pumps, density, g):
    return pump_pressure_rise(q / num_of_pumps, pump_speed, density, g)


def valve_1_loss_dp(q, valve_position, density, g):
    return valve_pressure_drop(q, valve_position, density, g)


def valve_2_loss_dp(q, valve_position, density, g):
    return valve_pressure_drop(q, valve_position, density, g)


def pipe_loss_dp(q, friction_factor, pipe_length, pipe_diameter, density):
    return darcy_weisbach_pressure_drop(
        q, friction_factor, pipe_length, pipe_diameter, density
    )


def calculate_delta_pressures(q, u):
    pump_speed, valve_1_position, valve_2_position = u
    return np.array(
        [
            STATIC_HEAD,
            pump_source_dp(q[1], pump_speed, NUM_OF_PUMPS, DENSITY, G),
            -pipe_loss_dp(q[2], FRICTION_FACTOR, PIPE_LENGTH, PIPE_DIAMETER, DENSITY)
            - STATIC_HEAD,
            -valve_1_loss_dp(q[3], valve_1_position, DENSITY, G),
            -valve_2_loss_dp(q[4], valve_2_position, DENSITY, G),
        ]
    )

In [5]:
"""STEADY STATE RHS"""


def rhs(z, u, scaling_factor=1.0):
    x = z * scaling_factor
    q = x[:N_BRANCHES]
    p = x[N_BRANCHES:]
    q = np.maximum(q, 0)
    p[0] = 0.0

    return np.concatenate(
        [
            INTERFACE_MATRIX @ q,
            INTERFACE_MATRIX.T @ p - calculate_delta_pressures(q, u),
        ]
    )

In [6]:
"""STEADT-STATE SOLVER"""

from scipy.optimize import fsolve


def solve_steady_state(u):
    q0 = np.array([0.3] * N_BRANCHES)
    p0 = np.array([1e5] * N_NODES)
    p0[0] = 0.0
    scaling_factor = np.concatenate([q0, p0])
    z0 = np.ones(N_BRANCHES + N_NODES)

    sol = fsolve(
        rhs,
        z0,
        args=(
            u,
            scaling_factor,
        ),

        xtol=1e-10,
        maxfev=int(1e8),
        full_output=True,
    )
    return sol[0] * scaling_factor, sol[1], sol[2]  # x, infodict, ier

In [7]:
"""DYNAMIC PARAMETERS"""

TAU_U1 = 1
TAU_U2 = 1
TAU_U3 = 1
CAPACITANCE_MATRIX = np.eye(N_NODES) * 1e-7
INDUCTANCE_MATRIX = np.eye(N_BRANCHES) * DENSITY * PIPE_LENGTH / PIPE_AREA


def calculate_controller_dynamics(t, u):
    setpoint = np.zeros_like(u)
    if t <= 10:
        setpoint[0] = 1.0
        setpoint[1] = 1.0
        setpoint[2] = 1.0
    if 10 < t <= 20:
        setpoint[0] = 0.5
        setpoint[1] = 1.0
        setpoint[2] = 1.0
    if 20 < t <= 40:
        setpoint[0] = 0.5
        setpoint[1] = 0.7
        setpoint[2] = 1.0

    if t > 40:
        setpoint[0] = 0.5
        setpoint[1] = 0.7
        setpoint[2] = 0.4
    return np.array(
        [
            (u[0] - setpoint[0]) / TAU_U1,
            (u[1] - setpoint[1]) / TAU_U2,
            (u[2] - setpoint[2]) / TAU_U3,
        ]
    )

In [8]:
"""QUASI-STATIC SIMULATION"""


from scipy.integrate import solve_ivp
import plotly.express as px
import pandas as pd

t_eval = np.arange(0, 50, 0.1)
TAU_U1 = 1.0
TAU_U2 = 1.0
TAU_U3 = 1.0


def ode(t, u):
    return -calculate_controller_dynamics(t, u)

u0 = [0.8, 0.5, 0.4]
sol = solve_ivp(
    ode,
    (t_eval[0], t_eval[-1]),
    u0,
    t_eval=t_eval,
    rtol=1e-9,
    atol=1e-9,
)

df = pd.DataFrame(
    {
        "t": sol.t,
        "pump_speed": sol.y[0],
        "valve_1_position": sol.y[1],
        "valve_2_position": sol.y[2],
    }
)
for index, row in df.iterrows():
    u = row[["pump_speed", "valve_1_position", "valve_2_position"]]
    sol = solve_steady_state(u)
    for i, q in enumerate(sol[0][:N_BRANCHES]):
        df.loc[index, f"q{i+1}"] = q
    for i, p in enumerate(sol[0][N_BRANCHES:]):
        df.loc[index, f"p{i+1}"] = p

df["Time (s)"] = df["t"]
df["Pump Speed (%)"] = df["pump_speed"] * 100
df["Valve 1 Position (%)"] = df["valve_1_position"] * 100
df["Valve 2 Position (%)"] = df["valve_2_position"] * 100
df["q1 (l/s)"] = df["q1"] * 1000
df["q2 (l/s)"] = df["q2"] * 1000
df["q3 (l/s)"] = df["q3"] * 1000
df["q4 (l/s)"] = df["q4"] * 1000
df["q5 (l/s)"] = df["q5"] * 1000
df["p1 (barg)"] = df["p1"] / 1e5
df["p2 (barg)"] = df["p2"] / 1e5
df["p3 (barg)"] = df["p3"] / 1e5
df["p4 (barg)"] = df["p4"] / 1e5
df.drop(columns=["pump_speed", "valve_1_position", "valve_2_position", "q1", "q2", "q3", "q4", "q5", "p1", "p2", "p3", "p4"], inplace=True)

In [9]:
fig = px.line(df, x="Time (s)", y=df.columns[1:])
fig.update_layout(template="plotly_dark", paper_bgcolor="#1f1f1f"
)